In [2]:
import time
import random
import requests
import selenium
from string import ascii_lowercase
from selenium.webdriver import Firefox, Chrome
from bs4 import BeautifulSoup
import pandas as pd
import string
import numpy as np
from selenium.webdriver.common.keys import Keys

In [3]:
browser = Chrome()

In [21]:
url='https://www.allrecipes.com/recipe/141833/thai-green-curry-chicken/'
browser.get(url)

In [84]:
time.sleep(2)

In [9]:
sel = 'ul'
ing_lists = browser.find_elements_by_css_selector(sel)

In [11]:
sel = 'ul#lst_ingredients_1'
ing_list_1 = browser.find_element_by_css_selector(sel)

In [12]:
ing_list_1.text.split('\n')

['1 pound skinless, boneless chicken breast halves - cut into 1 inch cubes',
 '1 tablespoon dark soy sauce',
 '1 tablespoon all-purpose flour',
 'Great Value All Purpose Flour, 32 oz',
 'In Stores Only',
 'SEE EVERYDAY LOW PRICE',
 'ADVERTISEMENT',
 '2 tablespoons cooking oil',
 '2 tablespoons green curry paste',
 '2 green onions with tops, chopped',
 '3 cloves garlic, peeled and chopped']

In [10]:
ing_lists.text.split('\n')

AttributeError: 'list' object has no attribute 'text'

In [13]:
sel = 'ul#lst_ingredients_2'
ing_list_2 = browser.find_element_by_css_selector(sel)

In [14]:
ing_list_2.text.split('\n')

['']

In [15]:
ing_lists = ing_list_1.text.split('\n') + ing_list_2.text.split('\n')

In [16]:
ingredients = []
for item in ing_lists:
    if item[0] in string.digits:
        ingredients.append(item)

IndexError: string index out of range

In [17]:
ingredients

[]

In [49]:
quantities = []
units = []
ingredient = []
for item in ingredients:
    quantities.append(item.split()[0])
    units.append(item.split()[1])
    ingredient.append(' '.join(item.split()[2:]))

In [1]:
units

NameError: name 'units' is not defined

In [22]:
def get_ingredients():
    all_items = []
    count = 1
    while True:
        try:
            sel = 'ul#lst_ingredients_{0}'.format(count)
            ing_list = browser.find_element_by_css_selector(sel)
            all_items += ing_list.text.split('\n')
            count += 1
        except:
            break
    ingredients = []
    for item in all_items:
        if item[0] in string.digits:
            ingredients.append(item)
    return ingredients

In [23]:
ing = get_ingredients()

In [24]:
ing

['1 pound skinless, boneless chicken breast halves - cut into 1 inch cubes',
 '1 tablespoon dark soy sauce',
 '1 tablespoon all-purpose flour',
 '2 tablespoons cooking oil',
 '2 tablespoons green curry paste',
 '2 green onions with tops, chopped',
 '3 cloves garlic, peeled and chopped',
 '1 teaspoon fresh ginger, peeled and finely chopped',
 '2 cups coconut milk',
 '1 tablespoon fish sauce',
 '1 tablespoon dark soy sauce',
 '2 tablespoons white sugar',
 '1/2 cup cilantro leaves, for garnish']

In [90]:
url='https://www.allrecipes.com/recipe/56094/thai-chicken-curry-in-coconut-milk'
browser.get(url)

In [91]:
find_ingredients()

['1 tablespoon vegetable oil',
 '1 teaspoon curry paste',
 '1 1/4 pounds skinless, boneless chicken breast meat - cut into strips',
 '1 onion, coarsely chopped',
 '1 red bell pepper, cut into strips',
 '1 tablespoon grated lemon zest',
 '1 cup light coconut milk',
 '1 tablespoon fish sauce',
 '1 tablespoon fresh lemon juice',
 '1/3 cup chopped fresh cilantro']

In [92]:
url='https://www.allrecipes.com/recipe/24789/gumbo-style-chicken-creole'
browser.get(url)

In [93]:
find_ingredients()

['1/4 cup vegetable oil',
 '1/4 cup all-purpose flour',
 '1 green bell pepper, chopped',
 '1 medium onion, chopped',
 '2 cups cooked chicken, chopped',
 '1 (14.5 ounce) can diced tomatoes with green chile peppers, with liquid',
 '1 (4.5 ounce) can sliced mushrooms, drained',
 '2 tablespoons chopped fresh parsley',
 '2 teaspoons Worcestershire sauce',
 '3 cloves garlic, minced',
 '1 teaspoon soy sauce',
 '1 teaspoon white sugar',
 '1/2 teaspoon salt',
 '1/2 teaspoon ground black pepper',
 '3 dashes hot sauce, or to taste',
 '1/2 cup chicken broth, or as needed (optional)']

In [25]:
def get_name():
    sel = 'h1#recipe-main-content'
    name = browser.find_element_by_css_selector(sel)
    return name.text

In [101]:
sel = 'div.rating-stars'
rating = browser.find_element_by_css_selector(sel)

In [102]:
rating.get_attribute('data-ratingstars')

'4.5330810546875'

In [26]:
def get_rating():
    sel = 'div.rating-stars'
    rating = browser.find_element_by_css_selector(sel)
    return float(rating.get_attribute('data-ratingstars'))

In [108]:
get_rating()

4.5330810546875

In [113]:
sel = 'ol.breadcrumbs li'
category_hierarchy = browser.find_elements_by_css_selector(sel)

In [115]:
category_hierarchy[0].text

'Home'

In [27]:
def get_categories():
    sel = 'ol.breadcrumbs li'
    categories = browser.find_elements_by_css_selector(sel)
    return [category.text for category in categories]

In [28]:
get_categories()

['Home', 'Recipes', 'World Cuisine', 'Asian', 'Thai']

In [5]:
def search_allrecipes(search_item, browser, typo_rate=0.1):
    """Goes to the Allrecipes.com homepage and searches for item specified by 'search_item'"""
    url='https://www.allrecipes.com'
    browser.get(url)
    sel = 'input#searchText'
    search_bar = browser.find_element_by_css_selector(sel)
    search_bar.click()
    type_item(search_item, search_bar, typo_rate)
    search_bar.send_keys('\n')

In [29]:
url='https://www.allrecipes.com'
browser.get(url)

In [30]:
sel = 'input#searchText'
search_bar = browser.find_element_by_css_selector(sel)
search_bar.click()

In [7]:
def type_item(item, field, typo_rate=0.1):
    for char in item:
        if np.random.random() < typo_rate:
            field.send_keys(make_typo(char))
            time.sleep(0.2 * np.random.random() + typo_rate)
            field.send_keys('\b')
        field.send_keys(char)
        time.sleep(0.05 * np.random.random() + typo_rate)

In [8]:
def make_typo(key, repeat=True):
    '''Returns a letter adjacent to (or identical to, if repeat=True) the input key.'''
    key = key.lower()
    row2 = list('qwertyuiop[')
    row3 = list("asdfghjkl;")
    row4 = list('zxcvbnm,.')
    adj_keys = [key] if repeat else []
    if key in row2:
        idx = row2.index(key)
        if idx > 0:
            adj_keys += [row2[idx-1] , row2[idx+1]] + row3[idx-1:idx+1]
        else:
            adj_keys += list('was')
    elif key in row3:
        idx = row3.index(key)
        if idx > 0:
            adj_keys += row2[idx:idx+2] + [row3[idx-1] , row3[idx+1]] + row4[idx-1:idx+1]
        else:
            adj_keys += list('qwsz')
    elif key in row4:
        idx = row4.index(key)
        if idx > 0:
            adj_keys += row3[idx:idx+2] + [row4[idx-1] , row4[idx+1]]
        else:
            adj_keys += list('asx')
    return np.random.choice(adj_keys)

In [57]:
search_bar.get_attribute('value')

'chicken enchiladas'

In [75]:
def clear_field(field):
    contents = field.get_attribute('value')
    for _ in contents:
        field.send_keys('\b')
        time.sleep(0.02 * np.random.random() + 0.05)

In [68]:
clear_field()

In [69]:
type_item('chicken pizza')

In [80]:
browser = Chrome()

In [9]:
search_allrecipes('chili', browser)

In [106]:
sel = 'a.toggle-similar'
sort = browser.find_element_by_css_selector(sel)
sort.click()

In [111]:
sel = 'a.toggle-similar ul.sort-nav'
sort_options = browser.find_elements_by_css_selector(sel)
sort_options

In [35]:
sel = 'article.fixed-recipe-card'
search_results = browser.find_elements_by_css_selector(sel)

In [38]:
sel = 'div.fixed-recipe-card__info'
info = search_results[22].find_element_by_css_selector(sel)
href = info.get_attribute('href')
name = info.text

In [39]:
name

'Elk Chili\n77\nA simple but tasty chili recipe using ground elk meat as the base. Best served with Mexican…\nBy JJHOME'

In [33]:
hrefs = []
names = []
for element in search_results:
    sel = 'div.fixed-recipe-card__info h3 a'
    info = element.find_element_by_css_selector(sel)
    hrefs.append(info.get_attribute('href'))
    names.append(info.text)
    print(info.text)

Boilermaker Tailgate Chili
Flatlander Chili
The Best Vegetarian Chili in the World
It's Chili by George!!
Easy Chili I
Debdoozie's Blue Ribbon Chili
Creamy White Chili
Just Like Wendy's® Chili
Frank's Spicy Alabama Onion Beer Chili
Terrific Turkey Chili
Smokin' Scovilles Turkey Chili
Beef, Bean, and Beer Chili
Washabinaros Chili
Chili I
Jeff's Hot Dog Chili
White Bean Chicken Chili
Best Damn Chili
My Chili
Paleo Chili
Quick Chili I
Chili Sauce


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.fixed-recipe-card__info h3 a"}
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=73.0.3683.20 (8e2b610813e167eee3619ac4ce6e42e3ec622017),platform=Mac OS X 10.14.3 x86_64)


In [32]:
len(names)

21

In [130]:
hrefs

['https://www.allrecipes.com/recipe/78299/boilermaker-tailgate-chili/?internalSource=hub%20recipe&referringContentType=Search',
 'https://www.allrecipes.com/recipe/13079/flatlander-chili/?internalSource=hub%20recipe&referringContentType=Search',
 'https://www.allrecipes.com/recipe/72508/the-best-vegetarian-chili-in-the-world/?internalSource=hub%20recipe&referringContentType=Search',
 'https://www.allrecipes.com/recipe/14985/its-chili-by-george/?internalSource=hub%20recipe&referringContentType=Search',
 'https://www.allrecipes.com/recipe/15401/easy-chili-i/?internalSource=hub%20recipe&referringContentType=Search',
 'https://www.allrecipes.com/recipe/16235/debdoozies-blue-ribbon-chili/?internalSource=hub%20recipe&referringContentType=Search',
 'https://www.allrecipes.com/recipe/229949/creamy-white-chili/?internalSource=hub%20recipe&referringContentType=Search',
 'https://www.allrecipes.com/recipe/231082/just-like-wendys-chili/?internalSource=hub%20recipe&referringContentType=Search',
 'h

In [49]:
browser.execute_script('''window.scrollTo(0, document.body.scrollHeight);
                       var lenOfPage=document.body.scrollHeight;return lenOfPage;''')

36271

In [46]:
def get_search_results(browser):
    '''
    Finds and returns all recipe names and hyperlink references on current browser page.
    '''
    sel = 'article.fixed-recipe-card'
    search_results = browser.find_elements_by_css_selector(sel)
    hrefs = []
    names = []
    for element in search_results:
        try:
            sel = 'div.fixed-recipe-card__info h3 a'
            info = element.find_element_by_css_selector(sel)
            href = info.get_attribute('href')
            hrefs.append(href.split('/?')[0])
            names.append(info.text)
        except:
            continue
    return names, hrefs

In [50]:
names, hrefs = get_search_results(browser)

In [51]:
len(names)

180

In [53]:
hrefs

['https://www.allrecipes.com/recipe/78299/boilermaker-tailgate-chili',
 'https://www.allrecipes.com/recipe/13079/flatlander-chili',
 'https://www.allrecipes.com/recipe/72508/the-best-vegetarian-chili-in-the-world',
 'https://www.allrecipes.com/recipe/14985/its-chili-by-george',
 'https://www.allrecipes.com/recipe/15401/easy-chili-i',
 'https://www.allrecipes.com/recipe/16235/debdoozies-blue-ribbon-chili',
 'https://www.allrecipes.com/recipe/229949/creamy-white-chili',
 'https://www.allrecipes.com/recipe/231082/just-like-wendys-chili',
 'https://www.allrecipes.com/recipe/70038/franks-spicy-alabama-onion-beer-chili',
 'https://www.allrecipes.com/recipe/62763/terrific-turkey-chili',
 'https://www.allrecipes.com/recipe/84760/smokin-scovilles-turkey-chili',
 'https://www.allrecipes.com/recipe/244558/beef-bean-and-beer-chili',
 'https://www.allrecipes.com/recipe/25711/washabinaros-chili',
 'https://www.allrecipes.com/recipe/16276/chili-i',
 'https://www.allrecipes.com/recipe/223538/jeffs-hot